# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to Ollama Python Client.
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Chat API : Ollama

Let's start with a Ollama local LLMs

In [7]:
import ollama

ollama.chat(model='zephyr', messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])

{'model': 'zephyr',
 'created_at': '2024-04-23T14:22:45.6259556Z',
 'message': {'role': 'assistant',
  'content': "The reason why the sky appears blue to us is due to the scattering of light in the Earth's atmosphere. When sunlight enters the Earth's atmosphere, it encounters various gases such as nitrogen and oxygen. These gases scatter (bounce) the light in all directions. The color of light that is scattered depends on its wavelength. Shorter wavelength colors like blue and violet are scattered more than longer wavelength colors like red and orange. This is why we see the sky appear blue. As the sun approaches the horizon, the amount of atmosphere between us and the sun increases, causing even more scattering and making the sky appear redder or more orange as a result."},
 'done': True,
 'total_duration': 57112296400,
 'load_duration': 12013795100,
 'prompt_eval_count': 25,
 'prompt_eval_duration': 6108911000,
 'eval_count': 142,
 'eval_duration': 38976465000}

In [12]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model = "zephyr",temperature = 0) # format = 'json' is also an argument 

llm.invoke("Tell me a joke")

AIMessage(content='Why did the tomato turn red?\n\nBecause it saw the salad dressing and didn\'t want to be dressed!\n\n(This is a play on words, as "dressed" can mean both "prepared with dressing" and "wearing clothes.")', response_metadata={'model': 'zephyr', 'created_at': '2024-04-23T14:36:32.1404959Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 13990722000, 'load_duration': 5822400, 'prompt_eval_duration': 444680000, 'eval_count': 56, 'eval_duration': 13532825000}, id='run-ff4405a2-627c-4b13-b32f-484583e95015-0')

In [13]:
llm.invoke("What is 1+1?")

AIMessage(content='The sum of one and one is two (2). In mathematical notation, this can be expressed as:\n\n1 + 1 = 2\n\nThis basic arithmetic operation is known as addition, and it forms the foundation for more complex mathematical concepts such as algebra, geometry, and calculus.', response_metadata={'model': 'zephyr', 'created_at': '2024-04-23T14:36:52.9593914Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 18791594000, 'load_duration': 3640600, 'prompt_eval_count': 17, 'prompt_eval_duration': 2555321000, 'eval_count': 64, 'eval_duration': 16227320000}, id='run-d303bfba-de6f-40ff-be81-c3dad3f96f4c-0')

### Prompt template

In [32]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

chat = ChatOllama(model = "zephyr",temperature = 0) # format = 'json' is also an argument 

In [33]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [34]:
print("The prompt template:",prompt_template.messages[0].prompt)

print("Input variables of prompt: ", prompt_template.messages[0].prompt.input_variables)

The prompt template: input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Input variables of prompt:  ['style', 'text']


In [35]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [36]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [37]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [38]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [39]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

print(customer_response.content)

"Ah, I'm seething that my blender lid flew off and splattered my kitchen walls with smoothie!" said the speaker in a calm and collected manner. "And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your assistance right now, matey!" The text was written using triple backticks, which is commonly used in programming languages to denote code snippets or command line input/output. However, this particular text appears to be a humorous and lighthearted statement, likely meant to evoke the spirit of pirates or sailors from the past. In any case, the speaker's request for help is clear and urgent, and their tone is polite and respectful.


In [40]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [41]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [42]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy there me hearties, 'tis ye, the customer, and we be lettin' ye know that the warranty don't cover the cost of cleanin' yer kitchen after ye misused yer blender by forgettin' to put the lid on before startin' it. Tough luck, matey! We'll see ye later!

(Note: The text is written in a humorous and exaggerated English Pirate style, but still maintains a polite tone.)


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [43]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [44]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [45]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [46]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(temperature=0.0, model="zephyr")
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "worth it for the extra features"
  ]
}


In [47]:
type(response.content)

str

In [56]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [48]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [49]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [50]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [53]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [54]:
response = chat.invoke(messages)

print(response.content)

```json
{
  "gift": false,
  "delivery_days": 2,
  "price_value": [
    "This leaf blower is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [55]:
output_dict = output_parser.parse(response.content)

print(output_dict)

{'gift': False, 'delivery_days': 2, 'price_value': ["This leaf blower is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [57]:
type(output_dict)

output_dict.get('delivery_days')

2